#### LangChain Essentials Course

# Getting Started with LangChain

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain, building a simple LLM-powered assistant. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

## Choosing your Model

This example is split into two versions, this version uses Ollama - allowing us to run our LLM locally without needing any external services or API keys. The [OpenAI version](https://github.com/aurelio-labs/agents-course/blob/main/04-langchain-ecosystem/01-langchain-essentials/01-langchain-intro-openai.ipynb) uses the OpenAI API and requires an OpenAI API key.

## Initializing Llama 3.2

We start by initializing the 1B parameter Llama 3.2 model, fine-tuned for instruction following. We `pull` the model from Ollama by switching to our terminal and executing:

```
ollama pull llama3.2:1b-instruct-fp16
```

Once the model has finished downloading, we initialize it in LangChain using the `ChatOllama` class:

In [1]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b-instruct-fp16"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

# initialize another LLM with temperature 0.9, this makes the LLM more creative
creative_llm = ChatOllama(temperature=0.9, model=model_name)

So the task at hand is to add any lyrics into the 'song' variable, then we want to create three core pieces from this song:

1. Song title
2. Song description
3. One additional verse in the song.

We will make all of this using a sequential chain, and then we will display it as:

> Original song:

> New song name:

> New song description:

> New verse:

> Where the new verse should be played in the song:

Here we can input our song to start us of with, currently this is using Viva La Vida by Coldplay

In [2]:
song = """
\
I used to rule the world \
Seas would rise when I gave the word \
Now in the morning, I sleep alone \
Sweep the streets I used to own \
I used to roll the dice \
Feel the fear in my enemy's eyes \
Listen as the crowd would sing \
Now the old king is dead, long live the king \
One minute, I held the key \
Next the walls were closed on me \
And I discovered that my castles stand \
Upon pillars of salt and pillars of sand \
\
I hear Jerusalem bells a-ringin' \
Roman Cavalry choirs are singin' \
Be my mirror, my sword and shield \
My missionaries in a foreign field \
For some reason, I can't explain \
Once you'd gone, there was never, never an honest word \
And that was when I ruled the world \
\
It was a wicked and wild wind \
Blew down the doors to let me in \
Shattered windows and the sound of drums \
People couldn't believe what I'd become \
Revolutionaries wait \
For my head on a silver plate \
Just a puppet on a lonely string \
Aw, who would ever wanna be king? \
\
I hear Jerusalem bells a-ringin' \
Roman Cavalry choirs are singing \
Be my mirror, my sword and shield \
My missionaries in a foreign field \
For some reason, I can't explain \
I know Saint Peter won't call my name \
Never an honest word \
But that was when I ruled the world \
\
Oh-oh-oh, oh-oh, oh \
Oh-oh-oh, oh-oh, oh \
Oh-oh-oh, oh-oh, oh \
Oh-oh-oh, oh-oh, oh \
Oh-oh-oh, oh-oh, oh \
\
I hear Jerusalem bells a-ringin' \
Roman Cavalry choirs are singin' \
Be my mirror, my sword and shield \
My missionaries in a foreign field \
For some reason I can't explain \
I know Saint Peter won't call my name \
Never an honest word \
But that was when I ruled the world \
"""

## Preparing our Prompts

LangChain comes with several prompt classes and methods for organizing or constructing our prompts. We will cover these in more detail in later examples, but for now we'll cover the essentials that we need here.

Prompts for chat agents are at a minimum broken up into three components, those are:

* System prompt: this provides the instructions to our LLM on how it must behave, what it's objective is, etc.

* User prompt: this is a user written input.

* AI prompt: this is the AI generated output. When representing a conversation, previous generations will be inserted back into the next prompt and become part of the broader _chat history_.

```
You are a helpful AI assistant, you will do XYZ.    | SYSTEM PROMPT

User: Hi, what is the capital of Australia?         | USER PROMPT
AI: It is Canberra                                  | AI PROMPT
User: When is the best time to visit?               | USER PROMPT
```

LangChain provides us with _templates_ for each of these prompt types. By using templates we can insert different inputs to the template, modifying the prompt based on the provided inputs.

Let's initialize our system and user prompt first:

In [4]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps generate song titles."
)

# the user prompt is provided by the user, in this case however the only dynamic
# input is the song
user_prompt = HumanMessagePromptTemplate.from_template(
    "Can you make a title for this song:\n\n{song}", input_variables=["song"]
)

We can display what our formatted human prompt would look like after inserting a value into the `song` parameter:

In [5]:
user_prompt.format(song="TEST STRING")

HumanMessage(content='Can you make a title for this song:\n\nTEST STRING', additional_kwargs={}, response_metadata={})

We have our system and user prompts, we can merge both into our full chat prompt using the `ChatPromptTemplate`:

In [6]:
from langchain.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

By default, the `ChatPromptTemplate` will read the `input_variables` from each of the prompt templates inserted and allow us to use those input variables when formatting the full chat prompt template:

In [7]:
print(first_prompt.format(song="TEST STRING"))

System: You are an AI assistant that helps generate song titles.
Human: Can you make a title for this song:

TEST STRING


`ChatPromptTemplate` also prefixes each individual message with it's role, ie `System:`, `Human:`, or `AI:`.

We can pull together our first prompt template and the `llm` object we defined earlier to create a simple `LLMChain` which chains together the steps **prompt formatting > llm generation > get output**.

In [8]:
from langchain.chains import LLMChain

chain_one = LLMChain(
    llm=creative_llm,  # for more creativity we use the LLM with temperature 0.9
    prompt=first_prompt, 
    output_key="song_title"  # specifies the output key for what our LLM generates
)

/var/folders/g7/qd2j76kn64ggrl4ym2vlvzwr0000gn/T/ipykernel_39114/2271991482.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(


Our first chain creates the song title, we can run it individually:

In [9]:
chain_one.invoke({"song": song})

{'song': "\nI used to rule the world Seas would rise when I gave the word Now in the morning, I sleep alone Sweep the streets I used to own I used to roll the dice Feel the fear in my enemy's eyes Listen as the crowd would sing Now the old king is dead, long live the king One minute, I held the key Next the walls were closed on me And I discovered that my castles stand Upon pillars of salt and pillars of sand I hear Jerusalem bells a-ringin' Roman Cavalry choirs are singin' Be my mirror, my sword and shield My missionaries in a foreign field For some reason, I can't explain Once you'd gone, there was never, never an honest word And that was when I ruled the world It was a wicked and wild wind Blew down the doors to let me in Shattered windows and the sound of drums People couldn't believe what I'd become Revolutionaries wait For my head on a silver plate Just a puppet on a lonely string Aw, who would ever wanna be king? I hear Jerusalem bells a-ringin' Roman Cavalry choirs are singing 

But we will actually chain this step with multiple other `LLMChain` steps. So, to continue, our next step is to summarize the lyrics using both the `song` and newly generated `song_title` values, from which we will output a new `summary` variable:

In [10]:
second_user_prompt = HumanMessagePromptTemplate.from_template(
    "Can you summarize the following song in 2 sentences (max 40 words) using "
    "the song title in the summary:\n\n{song}\n\n{song_title}",
    input_variables=["song", "song_title"]
)

second_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    second_user_prompt
])
# chain 2: inputs: song, song_title / outputs: summary
chain_two = LLMChain(
    llm=llm,  # we use the more deterministic LLM here
    prompt=second_prompt, 
    output_key="summary"
)

The third step will consume just our first `song` variable and then output a new `new_verse` variable.

In [11]:
third_user_prompt = HumanMessagePromptTemplate.from_template(
    "Can make one short verse for this song:\n\n{song}"
)

# prompt template 3: creating a new verse
third_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    third_user_prompt
])
# chain 3: inputs: song / output: new_verse
chain_three = LLMChain(
    llm=creative_llm, prompt=third_prompt,
    output_key="new_verse"
)

Finally, our fourth step will insert our new verse into the original set of lyrics. It consumes `new_verse` and `song`, then outputs `new_song`.

In [12]:
fourth_user_prompt = HumanMessagePromptTemplate.from_template(
    "where should the {new_verse} be played in the song {song}? Explicitly "
    "state which line of the original song this new verse should come after."
)

fourth_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    fourth_user_prompt
])
# chain 4: inputs: song, new_verse / outputs: new_song
chain_four = LLMChain(
    llm=llm,  # we need precision here so we use the more deterministic LLM
    prompt=fourth_prompt,
    output_key="new_song"
)

Now we have all our individual `LLMChain` steps ready we can _chain_ them together to create an end-to-end sequential chain. We use the `SequentialChain` to ensure each of our chains are run in sequence, which is required as most of our chains require input variables that are generated by previous chains.

In [13]:
from langchain.chains import SequentialChain

lyrics_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],  # our linked chains
    input_variables=["song"],  # the single input variable (used by our first chain)
    output_variables=["song_title", "summary","new_verse","new_song"],  # all of the outputs we want to return
    verbose=True  # to show AI intermediate steps
)

Now we can run our full chain using `invoke`, providing our single `song` input variable.

In [14]:
lyrics_chain.invoke({"song": song})



> Entering new SequentialChain chain...

> Finished chain.


{'song': "\nI used to rule the world Seas would rise when I gave the word Now in the morning, I sleep alone Sweep the streets I used to own I used to roll the dice Feel the fear in my enemy's eyes Listen as the crowd would sing Now the old king is dead, long live the king One minute, I held the key Next the walls were closed on me And I discovered that my castles stand Upon pillars of salt and pillars of sand I hear Jerusalem bells a-ringin' Roman Cavalry choirs are singin' Be my mirror, my sword and shield My missionaries in a foreign field For some reason, I can't explain Once you'd gone, there was never, never an honest word And that was when I ruled the world It was a wicked and wild wind Blew down the doors to let me in Shattered windows and the sound of drums People couldn't believe what I'd become Revolutionaries wait For my head on a silver plate Just a puppet on a lonely string Aw, who would ever wanna be king? I hear Jerusalem bells a-ringin' Roman Cavalry choirs are singing 